### The grand quest: make it actually work (4 points)

Your main task is to use some of the tricks you've learned on the network and analyze if you can improve __validation MAE__. Try __at least 3 options__ from the list below for a passing grade. Write a short report about what you have tried. More ideas = more bonus points. 

__Please be serious:__ " plot learning curves in MAE/epoch, compare models based on optimal performance, test one change at a time. You know the drill :)

You can use either pure __tensorflow__ or __keras__. Feel free to adapt the seminar code for your needs.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv("./Train_rev1.csv", index_col=None)

In [18]:
data['Log1pSalary'] = np.log1p(data['SalaryNormalized']).astype('float32')
text_columns = ["Title", "FullDescription"]
categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]
target_column = "Log1pSalary"

data[categorical_columns] = data[categorical_columns].fillna('NaN') # cast missing values to string "NaN"

data.sample(10)

,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SalaryRaw,SalaryNormalized,SourceName,Log1pSalary
157929,71125433,fabricator welder,fabricator welder days based west of leicester...,"Desford, Leicestershire",Desford,NaN,permanent,Other,Engineering Jobs,7.50 - 9.50/hour,16320,cv-library.co.uk,9.700208
45085,68538157,theatre lead reading,position – theatre lead salary **** k location...,Reading,Reading,full_time,NaN,JP Recruitment,Healthcare & Nursing Jobs,"50,000/Year",50000,staffnurse.com,10.819798
218703,72338145,accounts payable assistant,due to promotion from within an exciting oppor...,"Slough, Berkshire",Slough,NaN,permanent,Adecco,Accounting & Finance Jobs,21000/annum,21000,cv-library.co.uk,9.952325
35177,68234294,head chef ski season asap start,head chef ski season france small hotel amazin...,UK,UK,NaN,NaN,Platinum Recruitment Consultancy,Hospitality & Catering Jobs,750 per month + Great benefits,9000,caterer.com,9.105091
137584,70413292,management accountant uk,fintec recruit is seeking a management account...,"Daresbury, Cheshire, UK",Daresbury,NaN,NaN,NaN,"Energy, Oil & Gas Jobs",30000 - 35000 Per Annum,32500,oilandgasjobsearch.com,10.389026
104612,69573433,therapeutic services manager,18 . 5 hours per week the therapeutic services...,"Brighton, East Sussex",Brighton,part_time,NaN,Other,Charity & Voluntary Jobs,"30,011 - 32,800 pro rata",31405,charityjob.co.uk,10.354754
101558,69545791,ruby on rails developer,ruby developer my client is an award winning p...,London South East,South East London,NaN,permanent,Project Partners,IT Jobs,30000 - 65000 per annum + bonus & extensive be...,47500,cwjobs.co.uk,10.768506
154527,71014073,social housing property lawyer nq 1 years pqe ...,commercial firm in exeter is looking to recrui...,Devon,Devon,full_time,permanent,Other,Legal Jobs,"From 41,000 to 41,000 per year",41000,TotallyLegal,10.621352
145756,70693767,senior asset protection engineer,senior asset protection engineer **** **** lon...,London,London,NaN,permanent,Other,Engineering Jobs,"47,000 - 51,000",49000,justengineers.net,10.799596
56430,68682598,"industrial disease solicitor , west yorkshire",opening for a junior disease solicitor to join...,West Yorkshire Yorkshire,West Yorkshire,NaN,permanent,IPS Group,Legal Jobs,25000.00 - 30000.00 per annum,27500,totaljobs.com,10.221977


In [111]:
Counter(data.Company).most_common

<bound method Counter.most_common of Counter({'Other': 95650, 'NaN': 32430, 'UKStaffsearch': 4997, 'CVbrowser': 2977, 'London4Jobs': 2345, 'Hays': 1784, 'JAM Recruitment Ltd': 1122, 'Office Angels': 961, 'Jobsite Jobs': 932, 'Perfect Placement': 865, 'ARRAY': 847, 'JOBG8': 841, 'Matchtech Group plc.': 834, 'Penguin Recruitment': 752, 'Randstad': 748, 'Adecco': 701, 'Michael Page Finance': 634, 'Adecco Group': 592, 'BMS Sales Specialists LLP': 559, 'COREcruitment International': 551, 'Page Personnel Finance': 536, 'Capita Resourcing': 495, 'Michael Page Sales': 486, 'Matchtech': 474, 'Rise Technical Recruitment Ltd': 460, 'Experis IT': 446, 'ATA Recruitment': 438, 'Huxley Associates': 431, 'Interaction Recruitment': 428, 'Support Services Group': 419, 'Brookstreet UK': 417, 'Michael Page Marketing': 413, 'Real Staffing': 392, 'Spring Personnel': 392, 'Computer People': 375, '4Social Work': 373, 'SF Group': 373, 'Randstad Care': 348, 'Fresh Partnership': 347, 'Search Consultancy': 342, '

In [89]:
rubbish_columns = ['Id', 'LocationRaw', 'SalaryRaw', 'SourceName']
text_columns = ["Title", "FullDescription"]
categorical_columns = ["Category", "Company", "LocationNormalized", "ContractType", "ContractTime"]

# Preprocessing

In [122]:
from sklearn.base import TransformerMixin
import sklearn.preprocessing
from collections import defaultdict

In [121]:
class PreprocessGeneral(TransformerMixin):
    def fit(self, data):
        return self
        
    def transform(self, data):
        new_data = pd.DataFrame()
        for column in text_columns + categorical_columns:
            new_data[column] = data[column]
        new_data['Log1pSalary'] = np.log1p(data['SalaryNormalized']).astype('float32')
        new_data[categorical_columns] = data[categorical_columns].fillna('NaN')
        return new_data

In [178]:
le = sklearn.preprocessing.LabelEncoder()
le.fit(data["ContractType"] + data['ContractTime'])

LabelEncoder()

In [180]:
a = le.transform(data["ContractType"] + data['ContractTime'])

In [195]:
le.fit(np.concatenate([data['Category'], ['other']]))

LabelEncoder()

In [196]:
le.classes_

array(['Accounting & Finance Jobs', 'Admin Jobs',
       'Charity & Voluntary Jobs', 'Consultancy Jobs',
       'Creative & Design Jobs', 'Customer Services Jobs',
       'Domestic help & Cleaning Jobs', 'Energy, Oil & Gas Jobs',
       'Engineering Jobs', 'Graduate Jobs', 'HR & Recruitment Jobs',
       'Healthcare & Nursing Jobs', 'Hospitality & Catering Jobs',
       'IT Jobs', 'Legal Jobs', 'Logistics & Warehouse Jobs',
       'Maintenance Jobs', 'Manufacturing Jobs', 'Other/General Jobs',
       'PR, Advertising & Marketing Jobs', 'Part time Jobs',
       'Property Jobs', 'Retail Jobs', 'Sales Jobs',
       'Scientific & QA Jobs', 'Social work Jobs', 'Teaching Jobs',
       'Trade & Construction Jobs', 'Travel Jobs', 'other'], dtype=object)

In [193]:
le.transform(['other'])

ValueError: y contains new labels: ['other']

In [199]:
mean = data.groupby('preprocessed_' + 'Company')['Log1pSalary'].mean()

In [202]:
mean.get('ARRAY')

10.191086

## Categorical data

In [ ]:
class PreprocessCategorical(TransformerMixin):
    def __init__(min_company=40, min_city=20):
        self.le_category = sklearn.preprocessing.LabelEncoder()
        self.le_type = sklearn.preprocessing.LabelEncoder()
        self.city_counter = Counter()
        self.company_counter = Counter()
        self.min_company = min_company
        self.min_city = min_city
        
    def _fit_category(self, data):
        le.fit(np.concatenate([data['Category'], ['other']]))
        
    def _fit_type(self, data):
        self.le_type.fit(data['ContractTime'] + data['ContractType'])
        
    def _fit_city_company(self, data, what, minimum):
        keys = {value if count >= minimum else 'other' for value, count in Counter(data[what]).most_common()}
        data['preprocessed_' + what] = data[what].apply(lambda value: value if value in keys else 'other')
        self.mean[what] = data.groupby('preprocessed_' + what)['Log1pSalary'].mean()
        del data['preprocessed_' + what]
        
    def fit(self, data):
        self._fit_category(data)
        self._fit_city_company(data, 'LocationNormalized', self.min_city)
        self._fit_city_company(data, 'Company', self.min_company)
        self._fit_type(data)
        return self
    
    def _transform_category(self, data, new_data):
        new_data['Category'] = data['Category'].apply(lambda value: self.le_category.transform(value) 
                                                      if value in self.le_category.classes_ 
                                                      else self.le_category.transform('other'))
    
    def _transform_type(self, data, new_data):
        new_data['Contract'] = self.le_type.transform(data['ContractTime'] + data['ContractType'])
        
        
    def _transform_city_company(self, data, what, new_data):
        new_data[what] = data[what].apply(self.mean[what].get)
        
    def transform(self, data):
        new_data = pd.DataFrame()
        self._transform_category(data, new_data)
        self._transform_type(data, new_data)
        self._transform_city_company(data, 'LocationNormalized', new_data)
        self._transform_city_company(data, 'Company', new_data)
        for i 
        

## Text data

In [ ]:
import nltk

class PreprocessText(TransformerMixin):
    def __init__(self):
        tokenizer = nltk.tokenize.WordPunctTokenizer()
    
    def fit(self, data):
        for col in text_columns:
            data[col] = data[col].apply(lambda l: ' '.join(tokenizer.tokenize(str(l).lower())))
        
    def transform(self, data):

In [91]:
preprocess_data(data)

In [17]:
for col in categorical_columns:
    print(col, Counter(data[col]), '\n')

Category Counter({'IT Jobs': 38483, 'Engineering Jobs': 25174, 'Accounting & Finance Jobs': 21846, 'Healthcare & Nursing Jobs': 21076, 'Sales Jobs': 17272, 'Other/General Jobs': 17055, 'Teaching Jobs': 12637, 'Hospitality & Catering Jobs': 11351, 'PR, Advertising & Marketing Jobs': 8854, 'Trade & Construction Jobs': 8837, 'HR & Recruitment Jobs': 7713, 'Admin Jobs': 7614, 'Retail Jobs': 6584, 'Customer Services Jobs': 6063, 'Legal Jobs': 3939, 'Manufacturing Jobs': 3765, 'Logistics & Warehouse Jobs': 3633, 'Social work Jobs': 3455, 'Consultancy Jobs': 3263, 'Travel Jobs': 3126, 'Scientific & QA Jobs': 2489, 'Charity & Voluntary Jobs': 2332, 'Energy, Oil & Gas Jobs': 2255, 'Creative & Design Jobs': 1605, 'Maintenance Jobs': 1542, 'Graduate Jobs': 1331, 'Property Jobs': 1038, 'Domestic help & Cleaning Jobs': 291, 'Part time Jobs': 145}) 

Company Counter({'Other': 95650, 'NaN': 32430, 'UKStaffsearch': 4997, 'CVbrowser': 2977, 'London4Jobs': 2345, 'Hays': 1784, 'JAM Recruitment Ltd': 1122

In [4]:
import nltk
tokenizer = nltk.tokenize.WordPunctTokenizer()

for col in text_columns:
    data[col] = data[col].apply(lambda l: ' '.join(tokenizer.tokenize(str(l).lower())))

In [9]:
from collections import Counter
token_counts = Counter((' '.join(data[text_columns[0]])).split() + (' '.join(data[text_columns[1]])).split())

min_count = 10
# tokens from token_counts keys that had at least min_count occurrences throughout the dataset
tokens = [token for token, count in token_counts.items() if count >= min_count]

In [10]:
UNK, PAD = "UNK", "PAD"
tokens = [UNK, PAD] + sorted(tokens)
print("Vocabulary size:", len(tokens))


Vocabulary size: 34158


In [11]:
token_to_id = {token: index for index, token in enumerate(tokens)}
UNK_IX, PAD_IX = map(token_to_id.get, [UNK, PAD])

def as_matrix(sequences, max_len=None):
    """ Convert a list of tokens into a matrix with padding """
    if isinstance(sequences[0], str):
        sequences = list(map(str.split, sequences))
        
    max_len = min(max(map(len, sequences)), max_len or float('inf'))
    
    matrix = np.full((len(sequences), max_len), np.int32(PAD_IX))
    for i,seq in enumerate(sequences):
        row_ix = [token_to_id.get(word, UNK_IX) for word in seq[:max_len]]
        matrix[i, :len(row_ix)] = row_ix
    
    return matrix

In [12]:
from sklearn.feature_extraction import DictVectorizer

# we only consider top-1k most frequent companies to minimize memory usage
top_companies, top_counts = zip(*Counter(data['Company']).most_common(1000))
recognized_companies = set(top_companies)
data["Company"] = data["Company"].apply(lambda comp: comp if comp in recognized_companies else "Other")

categorical_vectorizer = DictVectorizer(dtype=np.float32, sparse=False)
categorical_vectorizer.fit(data[categorical_columns].apply(dict, axis=1))

DictVectorizer(dtype=<class 'numpy.float32'>, separator='=', sort=True,
        sparse=False)

In [13]:
from sklearn.model_selection import train_test_split

data_train, data_val = train_test_split(data, test_size=0.2, random_state=42)
data_train.index = range(len(data_train))
data_val.index = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))

Train size =  195814
Validation size =  48954


In [14]:
def make_batch(data, max_len=None, word_dropout=0):
    """
    Creates a keras-friendly dict from the batch data.
    :param word_dropout: replaces token index with UNK_IX with this probability
    :returns: a dict with {'title' : int64[batch, title_max_len]
    """
    batch = {}
    batch["Title"] = as_matrix(data["Title"].values, max_len)
    batch["FullDescription"] = as_matrix(data["FullDescription"].values, max_len)
    batch['Categorical'] = categorical_vectorizer.transform(data[categorical_columns].apply(dict, axis=1))
    
    if word_dropout != 0:
        batch["FullDescription"] = apply_word_dropout(batch["FullDescription"], 1. - word_dropout)
    
    if target_column in data.columns:
        batch[target_column] = data[target_column].values
    
    return batch

def apply_word_dropout(matrix, keep_prop, replace_with=UNK_IX, pad_ix=PAD_IX,):
    dropout_mask = np.random.choice(2, np.shape(matrix), p=[keep_prop, 1 - keep_prop])
    dropout_mask &= matrix != pad_ix
    return np.choose(dropout_mask, [matrix, np.full_like(matrix, replace_with)])

In [31]:
import keras
import keras.layers as L
import nltk.e

In [ ]:
def pre_train_embeddings(n_tokens=len(tokens), hid_size=64)
    emb = L.Embedding(n_tokens, hid_size)

In [27]:
def build_model(n_tokens=len(tokens), n_cat_features=len(categorical_vectorizer.vocabulary_), hid_size=100):
    """ Build a model that maps three data sources to a single linear output: predicted log1p(salary) """
    
    l_title = L.Input(shape=[None], name="Title")
    l_descr = L.Input(shape=[None], name="FullDescription")
    l_categ = L.Input(shape=[n_cat_features], name="Categorical")
    
    # Build your monster!
    
    
    emb_title = L.Embedding(n_tokens, hid_size)
    emb_discr = L.Embedding(n_tokens, 3*hid_size)
    emb_categ = L.Embedding(n_tokens, 3*hid_size)
    
    
    l_title_emb = emb(l_title)
    l_title_conv = L.Conv1D(filters=hid_size, kernel_size=2, activation='relu')(l_title_emb)
    l_title_out = L.GlobalMaxPool1D()(l_title_conv)
    
    l_descr_emp = emb(l_descr)
    l_descr_conv = L.Conv1D(filters=hid_size, kernel_size=2, activation='relu')(l_descr_emp)
    l_descr_out = L.GlobalMaxPool1D()(l_descr_conv)
    
    l_categ_out = L.Dense(units=2 * hid_size, activation='relu')(l_categ)
    
    l_combined = L.Concatenate()([l_title_out, l_descr_out, l_categ_out])
    l_combined_dense = L.Dense(2 * hid_size, activation='relu')(l_combined)
    
    output_layer = L.Dense(1)(l_combined_dense)
    
    model = keras.models.Model(inputs=[l_title, l_descr, l_categ], outputs=[output_layer])
    model.compile('adam', 'mean_squared_error', metrics=['mean_absolute_error'])
    return model
    

In [28]:
model = build_model()
model.summary()

dummy_pred = model.predict(make_batch(data_train[:100]))
dummy_loss = model.train_on_batch(make_batch(data_train[:100]), data_train['Log1pSalary'][:100])[0]
assert dummy_pred.shape == (100, 1)
assert len(np.unique(dummy_pred)) > 20, "model returns suspiciously few unique outputs. Check your initialization"
assert np.ndim(dummy_loss) == 0 and 0. <= dummy_loss <= 250., "make sure you minimize MSE"

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Title (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
FullDescription (InputLayer)    (None, None)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 64)     3212352     Title[0][0]                      
                                                                 FullDescription[0][0]            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)         

In [29]:
def iterate_minibatches(data, batch_size=256, shuffle=True, cycle=False, **kwargs):
    """ iterates minibatches of data in random order """
    while True:
        indices = np.arange(len(data))
        if shuffle:
            indices = np.random.permutation(indices)

        for start in range(0, len(indices), batch_size):
            batch = make_batch(data.iloc[indices[start : start + batch_size]], **kwargs)
            target = batch.pop(target_column)
            yield batch, target
        
        if not cycle: break

In [ ]:
emb = L.embeddings()

In [37]:
data_train['FullDescription'], data_train['Title']

0                                     senior support worker
1         statistical / mathematical modeller in pharmac...
2                              emc team leader ( def stan )
3                      core java engineer london , big data
4                                                    driver
5                         vn **** ait rf technicians x ****
6         general manager mediterranean restaurant marke...
7                     stock control opperative / van driver
8         system tester linux , javascript , html , css ...
9                      customer service adviser – part time
10                      children s support worker wiltshire
11                                             bms engineer
12                                   recruitment consultant
13             business travel consultant , leeds , ls ****
14                          bdm restaurant fit out national
15                                        android developer
16        c / aspnet mvc developer migra

### A short report

Please tell us what you did and how did it work.

`<YOUR_TEXT_HERE>`, i guess...

## Recommended options

#### A) CNN architecture

All the tricks you know about dense and convolutional neural networks apply here as well.
* Dropout. Nuff said.
* Batch Norm. This time it's `L.BatchNormalization`
* Parallel convolution layers. The idea is that you apply several nn.Conv1d to the same embeddings and concatenate output channels.
* More layers, more neurons, ya know...


#### B) Play with pooling

There's more than one way to perform pooling:
* Max over time - our `L.GlobalMaxPool1D`
* Average over time (excluding PAD)
* Softmax-pooling:
$$ out_{i, t} = \sum_t {h_{i,t} \cdot {{e ^ {h_{i, t}}} \over \sum_\tau e ^ {h_{j, \tau}} } }$$

* Attentive pooling
$$ out_{i, t} = \sum_t {h_{i,t} \cdot Attn(h_t)}$$

, where $$ Attn(h_t) = {{e ^ {NN_{attn}(h_t)}} \over \sum_\tau e ^ {NN_{attn}(h_\tau)}}  $$
and $NN_{attn}$ is a dense layer.

The optimal score is usually achieved by concatenating several different poolings, including several attentive pooling with different $NN_{attn}$ (aka multi-headed attention).

The catch is that keras layers do not inlude those toys. You will have to [write your own keras layer](https://keras.io/layers/writing-your-own-keras-layers/). Or use pure tensorflow, it might even be easier :)

#### C) Fun with words

It's not always a good idea to train embeddings from scratch. Here's a few tricks:

* Use a pre-trained embeddings from `gensim.downloader.load`. See last lecture.
* Start with pre-trained embeddings, then fine-tune them with gradient descent. You may or may not want to use __`.get_keras_embedding()`__ method for word2vec
* Use the same embedding matrix in title and desc vectorizer


#### D) Going recurrent

We've already learned that recurrent networks can do cool stuff in sequence modelling. Turns out, they're not useless for classification as well. With some tricks of course..

* Like convolutional layers, LSTM should be pooled into a fixed-size vector with some of the poolings.
* Since you know all the text in advance, use bidirectional RNN
  * Run one LSTM from left to right
  * Run another in parallel from right to left 
  * Concatenate their output sequences along unit axis (dim=-1)

* It might be good idea to mix convolutions and recurrent layers differently for title and description


#### E) Optimizing seriously

* You don't necessarily need 100 epochs. Use early stopping. If you've never done this before, take a look at [early stopping callback](https://keras.io/callbacks/#earlystopping).
  * In short, train until you notice that validation
  * Maintain the best-on-validation snapshot via `model.save(file_name)`
  * Plotting learning curves is usually a good idea
  
Good luck! And may the force be with you!